In [ ]:
# Generate or load a synthetic time series dataset and preprocess it for LSTM input.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Generate synthetic time series data
def create_time_series(n_samples, n_timestamps, n_classes):
    X = []
    y = []
    for _ in range(n_samples):
        class_label = np.random.randint(0, n_classes)
        time_series = np.sin(np.linspace(0, 2 * np.pi, n_timestamps) + class_label)
        X.append(time_series)
        y.append(class_label)
    return np.array(X), np.array(y)

# Parameters
n_samples = 1000
n_timestamps = 50
n_classes = 3

# Create dataset
X, y = create_time_series(n_samples, n_timestamps, n_classes)

# Reshape for LSTM input: (samples, timesteps, features)
X = X[..., np.newaxis]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

In [ ]:
# Define an LSTM model architecture for classifying time series data into categories.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=n_classes)
y_test = to_categorical(y_test, num_classes=n_classes)

# Build the LSTM model
model = Sequential([
    LSTM(50, activation='tanh', input_shape=(n_timestamps, 1)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(n_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the LSTM model on the time series dataset and monitor validation accuracy.

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Evaluate the LSTM model on the test dataset and visualize training/validation performance.

# Evaluate test accuracy and loss
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

# Plot training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.show()

In [ ]:
# Test the LSTM model on new time series data to predict class labels.

# Generate a new time series sample
new_sample = np.sin(np.linspace(0, 2 * np.pi, n_timestamps) + 1)[..., np.newaxis]  # Class 1
new_sample = new_sample[np.newaxis, ...]

# Predict the class
prediction = model.predict(new_sample)
predicted_class = np.argmax(prediction)
print("Predicted Class:", predicted_class)

# Visualize the new sample
plt.figure(figsize=(6, 4))
plt.plot(new_sample[0], label="New Sample (Class 1)")
plt.title("New Time Series Sample")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.legend()
plt.show()